<a href="https://colab.research.google.com/github/GaiMarina/AB_testing/blob/main/8_AB_HomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 8. Python применение статистических критериев на практике - пайплайн оценки A/B

---
На сайте запущен А/В тест с целью увеличить доход. 

В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

---

In [54]:
from math import asin
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power
import plotly.express as px

In [55]:
# Непрерывная метрика.

def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,    # Колонка, чтобы производить расчет эффекта.
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]  # Выбираем размер.
    
    control_mean = control.loc[:, column].mean()    # Среднее по контролю
    treatment_mean = treatment.loc[:, column].mean()  # Среднеей по тесту
    
    control_std = control.loc[:, column].std(ddof=1)  # STD по контролю, по тесту
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    # Вместо встроенной ф-и effectsize_smd: Нормированную разницу средних на групповое SD.
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,   # Мощность эксперимента.
                               nobs1=size,  # Кол-во наблюдений, вытащили вначале.
                               alpha=.05,
                               power=None,    # power - None, т.к. его ищем.
                               ratio=1) # 1, т.к. один датафрейм
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

## Рассмотрим кейс с рекламной выручкой

In [3]:
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Data/gb_sem_8_hm.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [5]:
df.shape

(10000, 3)

In [6]:
df.USER_ID.nunique()

6324

In [7]:
# Убрали дубликаты. Обновили индексы на индексы по умолчанию.

df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})
df

,USER_ID,VARIANT_NAME,REVENUE
0,2,control,0.00
1,3,control,0.00
2,3,variant,0.00
3,4,variant,0.00
4,5,variant,0.00
...,...,...,...
7860,9996,control,0.00
7861,9996,variant,6.46
7862,9998,control,0.00
7863,10000,control,0.00


In [8]:
df.shape

(7865, 3)

In [11]:
# У 1541 айдишников - по 2 значения.
# Посчитали количество вариантов у юзер_айди.

df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [27]:
# Массив numpy из уникальных айдишников. Если у айди > чем 1 вариант, он исключается.

unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1') 
 .USER_ID
 .values
 )
unique_ids

array([   2,    4,    5, ..., 9993, 9995, 9998])

In [36]:
# Новый датафрейм на основе уникальных айдишников.
# deep copy не зависима от оригинала.

df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)
df_new, df

(      USER_ID VARIANT_NAME  REVENUE
 0           2      control      0.0
 3           4      variant      0.0
 4           5      variant      0.0
 5           6      variant      0.0
 6           9      variant      0.0
 ...       ...          ...      ...
 7856     9990      variant      0.0
 7857     9992      control      0.0
 7858     9993      control      0.0
 7859     9995      variant      0.0
 7862     9998      control      0.0
 
 [4783 rows x 3 columns],       USER_ID VARIANT_NAME  REVENUE
 0           2      control     0.00
 1           3      control     0.00
 2           3      variant     0.00
 3           4      variant     0.00
 4           5      variant     0.00
 ...       ...          ...      ...
 7860     9996      control     0.00
 7861     9996      variant     6.46
 7862     9998      control     0.00
 7863    10000      control     0.00
 7864    10000      variant     0.00
 
 [7865 rows x 3 columns])

In [65]:
df_new.dtypes

USER_ID           int64
VARIANT_NAME     object
REVENUE         float64
dtype: object

In [37]:
df_new.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [38]:
# Делим данные на тест/контроль

control = df_new[df_new['VARIANT_NAME'] == 'control'].copy(deep=True)
test = df_new[df_new['VARIANT_NAME'] == 'variant'].copy(deep=True)
control, test

(      USER_ID VARIANT_NAME  REVENUE
 0           2      control      0.0
 9          11      control      0.0
 10         12      control      0.0
 11         13      control      0.0
 17         21      control      0.0
 ...       ...          ...      ...
 7850     9980      control      0.0
 7855     9989      control      0.0
 7857     9992      control      0.0
 7858     9993      control      0.0
 7862     9998      control      0.0
 
 [2390 rows x 3 columns],       USER_ID VARIANT_NAME  REVENUE
 3           4      variant      0.0
 4           5      variant      0.0
 5           6      variant      0.0
 6           9      variant      0.0
 12         15      variant      0.0
 ...       ...          ...      ...
 7844     9975      variant      0.0
 7853     9983      variant      0.0
 7854     9986      variant      0.0
 7856     9990      variant      0.0
 7859     9995      variant      0.0
 
 [2393 rows x 3 columns])

In [49]:
# Смотрим отдельно описание по обеим группам.

control.describe(), '---' * 10, test.describe()

(           USER_ID      REVENUE
 count  2390.000000  2390.000000
 mean   5020.881590     0.196887
 std    2904.850992     4.172201
 min       2.000000     0.000000
 25%    2517.250000     0.000000
 50%    5012.500000     0.000000
 75%    7616.000000     0.000000
 max    9998.000000   196.010000,
 '------------------------------',
            USER_ID      REVENUE
 count  2393.000000  2393.000000
 mean   4967.943168     0.074935
 std    2892.745368     0.858207
 min       4.000000     0.000000
 25%    2435.000000     0.000000
 50%    4955.000000     0.000000
 75%    7379.000000     0.000000
 max    9995.000000    23.040000)

---

#### ***Предварительно тест показывает ухудшение метрики. Значительное снижение показателя "максимальный доход".***
---

In [51]:
fig = px.histogram(df_new, x="REVENUE",
                   color='VARIANT_NAME', barmode='group',
                   height=400)
fig.show()

---

#### ***Метрика revenue очень разряженная. Большинство людей ничего не покупает => большинство данных - у 0. У контрольной группы - тяжелый хвост (значение около 196). <br>Скорее всего здесь не распределение Бернулли, а отличное от него - экспоненциальное распр-ние.***
---

In [58]:
# Разница в тесте и контроле на всех пользователях. 

continious_result(control, test, column='REVENUE')

100%|██████████| 10000/10000 [00:04<00:00, 2061.62it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.040483,0.166662,0.712143,"[-0.004, 0.328]",0.122354


In [67]:
# Разница в тесте и контроле на тех, с кого был доход.

continious_result(control[control.REVENUE != 0], 
                       test[test.REVENUE != 0],
                       column='REVENUE')

100%|██████████| 10000/10000 [00:02<00:00, 3708.97it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.230414,0.223978,0.779569,"[-0.584, 12.732]",4.451616


---

***Данный эксперимент успешным не является. Статистически значимых различий между контрольной и тестовой группами нет. Не имеет смысла "раскатывать" результаты данного эксперимента на пользователей.***

---
alpha, beta завышены, 

слишком маленькая мощность эксперимента (1 - beta), 

0 входит в доверительный интервал. 

Верна нулевая гипотеза H0 об отсутствии различий между группами.